In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('data.csv')
df.head()

,review,sentiment
0,Every great gangster movie has under-currents ...,positive
1,"I just saw this film last night, and I have to...",positive
2,This film is mildly entertaining if one neglec...,negative
3,Quentin Tarantino's partner in crime Roger Ava...,negative
4,I sat through this on TV hoping because of the...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
0,every great gangster movie under current human...,positive
1,saw film last night say loved every minute tak...,positive
2,film mildly entertaining one neglect acknowled...,negative
3,quentin tarantino s partner crime roger avary ...,negative
4,sat tv hoping name would worth time but dear g...,negative


In [7]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,every great gangster movie under current human...,1
1,saw film last night say loved every minute tak...,1
2,film mildly entertaining one neglect acknowled...,0
3,quentin tarantino s partner crime roger avary ...,0
4,sat tv hoping name would worth time but dear g...,0


In [10]:
#what is the purpose of count vectorizer?
# CountVectorizer converts a collection of text documents to a matrix of token counts. 
# It is used to convert the text data into a format that can be used by machine learning algorithms.
# It creates a sparse matrix where each row represents a document and each column represents a token (word).
# The value in each cell of the matrix represents the count of the token in the document.
# The CountVectorizer is used to convert the text data into a format that can be used by machine learning algorithms.

vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [ ]:
#if I want to see 1st record of the vectorizer
#other options to count vectorizer
#vectorizer.get_feature_names_out() 
X[0].toarray()

array([[0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 8, 0, 3, 1, 0, 0, 0, 0, 0,
        0, 2, 1, 1, 0, 5, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 3, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 4, 1, 0, 2, 1, 0, 0,
        0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0]])

In [16]:
vectorizer.get_feature_names_out() 

array(['acting', 'actor', 'actually', 'all', 'almost', 'also', 'and',
       'another', 'back', 'bad', 'best', 'better', 'book', 'br', 'can',
       'character', 'come', 'could', 'day', 'director', 'done', 'end',
       'even', 'ever', 'every', 'fact', 'feel', 'film', 'find', 'first',
       'get', 'give', 'go', 'going', 'good', 'great', 'guy', 'horror',
       'interesting', 'it', 'know', 'life', 'like', 'little', 'long',
       'look', 'lot', 'love', 'made', 'make', 'man', 'many', 'movie',
       'much', 'must', 'never', 'new', 'nothing', 'old', 'one', 'part',
       'people', 'performance', 'play', 'plot', 'pretty', 'quite', 'real',
       'really', 'role', 'say', 'scene', 'see', 'seems', 'seen', 'show',
       'something', 'still', 'story', 'take', 'that', 'the', 'there',
       'thing', 'think', 'this', 'though', 'time', 'two', 'want', 'watch',
       'watching', 'way', 'well', 'whole', 'woman', 'work', 'world',
       'would', 'year'], dtype=object)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [7]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/souvikpaul425/yt_mlops_capstone_project.mlflow/')
dagshub.init(repo_owner='souvikpaul425', repo_name='yt_mlops_capstone_project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline V2")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\souvik\anaconda3\envs\atlas\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4e6f147b-df27-4ef6-b4ad-e8389487ab8d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=670008f7f463f53ae2abedcd5add6e69fe6113a79fd672daf00f1113b05db471




Accessing as souvikpaul425

Initialized MLflow to track repo "souvikpaul425/yt_mlops_capstone_project"

Repository souvikpaul425/yt_mlops_capstone_project initialized!

2025/05/28 14:49:49 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline V2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/815f9a20411b4f04bf31d4242d566725', creation_time=1748423990194, experiment_id='4', last_update_time=1748423990194, lifecycle_stage='active', name='Logistic Regression Baseline V2', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run(): 
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-05-28 14:58:13,734 - INFO - Starting MLflow run...
2025-05-28 14:58:14,106 - INFO - Logging preprocessing parameters...
2025-05-28 14:58:15,168 - INFO - Initializing Logistic Regression model...
2025-05-28 14:58:15,169 - INFO - Fitting the model...
2025-05-28 14:58:15,226 - INFO - Model training complete.
2025-05-28 14:58:15,227 - INFO - Logging model parameters...
2025-05-28 14:58:15,551 - INFO - Making predictions...
2025-05-28 14:58:15,551 - INFO - Calculating evaluation metrics...
2025-05-28 14:58:15,566 - INFO - Logging evaluation metrics...
2025-05-28 14:58:16,971 - INFO - Saving and logging the model...
2025/05/28 14:58:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-05-28 14:58:37,275 - INFO - Model training and logging completed in 23.17 seconds.
2025-05-28 14:58:37,277 - INFO - Accuracy: 0.66
2025-05-28 14:58:37,277 - INFO - Precision: 0

🏃 View run painted-eel-603 at: https://dagshub.com/souvikpaul425/yt_mlops_capstone_project.mlflow/#/experiments/4/runs/a34122742b844c36ae19535c876592f0
🧪 View experiment at: https://dagshub.com/souvikpaul425/yt_mlops_capstone_project.mlflow/#/experiments/4


In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run(): 
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.20)

        logging.info("Defining hyperparameter grid for Logistic Regression...")
        param_grid = {
            'penalty': ['l1', 'l2'],
            'C': [0.01, 0.1, 1, 10],
            'solver': ['liblinear'],  # 'liblinear' supports both l1 and l2
            'max_iter': [1000]
        }

        logging.info("Initializing GridSearchCV...")
        grid_search = GridSearchCV(
            LogisticRegression(),
            param_grid,
            cv=5,
            scoring='f1',
            n_jobs=-1
        )

        logging.info("Fitting the model with GridSearchCV...")
        grid_search.fit(X_train, y_train)
        logging.info("Grid search complete.")

        best_model = grid_search.best_estimator_

        logging.info("Logging best hyperparameters from grid search...")
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_param("model", "Logistic Regression (GridSearchCV)")

        logging.info("Making predictions with the best model...")
        y_pred = best_model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the best model...")
        mlflow.sklearn.log_model(best_model, "model")

        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-05-28 15:06:18,252 - INFO - Starting MLflow run...
2025-05-28 15:06:18,858 - INFO - Logging preprocessing parameters...
2025-05-28 15:06:19,911 - INFO - Defining hyperparameter grid for Logistic Regression...
2025-05-28 15:06:19,911 - INFO - Initializing GridSearchCV...
2025-05-28 15:06:19,911 - INFO - Fitting the model with GridSearchCV...
2025-05-28 15:06:36,761 - INFO - Grid search complete.
2025-05-28 15:06:36,761 - INFO - Logging best hyperparameters from grid search...
2025-05-28 15:06:37,475 - INFO - Making predictions with the best model...
2025-05-28 15:06:37,475 - INFO - Calculating evaluation metrics...
2025-05-28 15:06:37,498 - INFO - Logging evaluation metrics...
2025-05-28 15:06:38,895 - INFO - Saving and logging the best model...
2025/05/28 15:06:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-05-28 15:06:47,979 - INFO - Model train

🏃 View run amusing-shoat-37 at: https://dagshub.com/souvikpaul425/yt_mlops_capstone_project.mlflow/#/experiments/4/runs/4115e9f0989f49c18c5cc8a168027774
🧪 View experiment at: https://dagshub.com/souvikpaul425/yt_mlops_capstone_project.mlflow/#/experiments/4


In [14]:
grid_search.best_params_

{'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}

In [ ]:
2025-05-28 15:06:47,980 - INFO - Accuracy: 0.74
2025-05-28 15:06:47,981 - INFO - Precision: 0.7727272727272727
2025-05-28 15:06:47,982 - INFO - Recall: 0.68
2025-05-28 15:06:47,983 - INFO - F1 Score: 0.723404255319149